In [ ]:
pip install flask flask-cors requests pyngrok

In [ ]:
pip install transformers accelerate

In [ ]:
# Install dependencies
!apt-get update && apt-get install -y curl

# Download and install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Download the model (this might take some time)
!ollama pull deepseek-r1:1.5b

# Start Ollama server in background
!ollama serve &

# Wait for server to start
import time
time.sleep(10)

# Main chatbot
import requests
import json
from google.colab import output
output.clear()

class OllamaAPI:
    def __init__(self, base_url="http://localhost:11434"):
        self.base_url = base_url

    def generate(self, model="deepseek-r1:1.5b", prompt="", stream=False):
        """
        Generate response using Ollama API
        """
        url = f"{self.base_url}/api/generate"
        data = {
            "model": model,
            "prompt": prompt,
            "stream": stream,
            "options": {
                "temperature": 0.7,
                "top_p": 0.9,
                "top_k": 40
            }
        }

        try:
            response = requests.post(url, json=data)
            response.raise_for_status()
            return response.json()['response']
        except requests.exceptions.RequestException as e:
            return f"Error: {str(e)}"

def main():
    print("Initializing DeepSeek chatbot di Google Colab...")
    print("Memastikan Ollama server berjalan...")

    # Initialize Ollama API
    ollama = OllamaAPI()

    print("\nChatbot siap! (ketik 'exit' untuk keluar)")
    print("----------------------------------------")

    while True:
        user_input = input("\nAnda: ")

        if user_input.lower() == 'exit':
            print("Chatbot: Terima kasih telah mengobrol! Sampai jumpa!")
            break

        # Generate response
        response = ollama.generate(prompt=user_input)
        print(f"\nChatbot: {response}")

if __name__ == "__main__":
    main()

In [ ]:
#Stop above this, then run the below code to get the ngrok link

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import requests
from pyngrok import ngrok
import time
import subprocess
import os

app = Flask(__name__)
CORS(app)

class OllamaAPI:
    def __init__(self, base_url="http://127.0.0.1:11434"):
        self.base_url = base_url
        self._ensure_ollama_running()

    def _ensure_ollama_running(self):
        max_retries = 3
        for attempt in range(max_retries):
            try:
                requests.get(f"{self.base_url}/api/tags")
                print("Ollama server is already running!")
                return
            except requests.exceptions.ConnectionError:
                if attempt == 0:
                    print("Starting Ollama server...")
                    subprocess.Popen(["ollama", "serve"],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)
                time.sleep(5)  
        raise Exception("Failed to start Ollama server after multiple attempts")

    def generate(self, prompt, model="deepseek-r1:1.5b"):
        if not prompt.strip():
            return "Please provide a valid input."

        url = f"{self.base_url}/api/generate"
        data = {
            "model": model,
            "prompt": prompt,
            "stream": False,  # Disable streaming for simpler handling
            "options": {
                "temperature": 0.7,
                "top_p": 0.9,
                "top_k": 40,
                "timeout": 30  # timeout setting
            }
        }

        try:
            response = requests.post(url, json=data, timeout=60)  # Increased timeout
            response.raise_for_status()
            return response.json().get('response', 'No response generated')
        except requests.exceptions.Timeout:
            return "Request timed out. Please try again."
        except Exception as e:
            return f"Error: {str(e)}"

# Initial Ollama API
ollama = OllamaAPI()

@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.json
        prompt = data.get('prompt', '').strip()

        if not prompt:
            return jsonify({'error': 'No prompt provided'}), 400

        response = ollama.generate(prompt)
        return jsonify({'response': response})
    except Exception as e:
        return jsonify({'error': f'Server error: {str(e)}'}), 500

def main():
    # Ngrok
    ngrok.set_auth_token("YOUR-NGROK-AUTH-TOKEN")
    public_url = ngrok.connect(5000)

    print(f"\n=== SALIN URL INI KE CLIENT.PY DI DEVICE ANDA ===")
    print(f"URL: {public_url}")
    print("===============================================\n")

    # Run Flask app
    app.run(port=5000)

if __name__ == '__main__':
    main()

In [26]:
!pkill ollama